# Notebook purpose:

- Demonstrate how to set up a lattice based sampling experiment. 
- Generate and pass on a data structure which contains:
    - sample information
        - all possible sample compositions which satisfy **Sum(volume fraction) = 1**
        - component names and properties
        - extensive properties for each sample (mass, volume, moles)
        

In [1]:
import pandas as pd

import importlib
import setup_functions
importlib.reload(setup_functions)

C:\Users\lacho\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\lacho\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


<module 'setup_functions' from 'C:\\Users\\lacho\\Documents\\Pozzo-RG-OT2\\OT2Protocols\\code-collab\\database\\setup_functions.py'>

# 1. 

- experiment plan .csv $\rightarrow$ dictionary. get_experiment_plan makes use of ast module. 
- chemical inventory .xls $\rightarrow$ pandas dataframe.


In [5]:
experiment_plan_filepath = 'experiment_plan.csv'
experiment_plan_dict = setup_functions.get_experiment_plan(experiment_plan_filepath)
chemical_db_df = pd.read_excel('chemical_inventory.xlsx')
chemical_db_df

,Chemical Name,Chemical Abbreviation,Molecular Weight (g/mol),Density (g/mL),InChI=,InChI Key (ID),Chemical Category,Canonical SMILES String,Molecular Formula,PubChem ID,Catalog description,Synonyms,Catalog number,Purchase URL,Primary Information Source
0,Water,Water,18.015,0.995,1S/H2O/h1H2,XLYOFNOQVPJJNP-UHFFFAOYSA-N,solvent,O,H2O,962,NaN,NaN,NaN,NaN,NaN
1,Sodium Chloride,NaCl,58.440,2.165,1S/ClH.Na/h1H;/q;+1/p-1,FAPWRFPIFSIZLT-UHFFFAOYSA-M,"inorganic, halide",[Na+].[Cl-],NaCl,5234,NaN,NaN,NaN,NaN,NaN
2,Butenedioic acid,Butenedioicacid,116.070,1.500,"1S/C4H4O4/c5-3(6)1-2-4(7)8/h1-2H,(H,5,6)(H,7,8)",VZCYOOQTPOCHFL-UHFFFAOYSA-N,"hbd, acid",C(=CC(=O)O)C(=O)O,C4H4O4,723,NaN,NaN,NaN,NaN,NaN
3,Choline chloride,chcl,139.620,NaN,"1S/C5H14NO.ClH/c1-6(2,3)4-5-7;/h7H,4-5H2,1-3H3...",SGMZJAMFUVOLNK-UHFFFAOYSA-M,"hba, organic, halide",C[N+](C)(C)CCO.[Cl-],C5H14ClNO,6209,NaN,NaN,NaN,NaN,NaN
4,Gold (III) Chloride Trihydrate,haucl4,393.830,3.900,1S/Au.4ClH.3H2O/h;4*1H;3*1H2/q+3;;;;;;;/p-3,XYYVDQWGDNRQDA-UHFFFAOYSA-K,"acid, inorganic",O.O.O.Cl.Cl[Au](Cl)Cl,AuCl4H7O3,44134746,NaN,NaN,NaN,NaN,NaN


In [113]:
experiment_plan_dict

{'Protocol Version': 'testing',
 'Experimenter': 'Kacper Lachowski',
 'Project tag': 'testing',
 'Experiment tag': 'test1',
 'Sample volume (uL)': 1000,
 'Stock order': ['hba-stock', 'hbd-stock', 'Water'],
 'Component names': ['NaCl', 'Butenedioicacid', 'chcl', 'Water'],
 'Concentration types': ['molarity', 'molarity', 'molarity', 'volf'],
 'Concentration linspace [min, max, n]': [[0.001, 1, 20],
  [0.001, 1, 20],
  [1, 40, 20]],
 'Left pipette': 'P1000_Single',
 'Right pipette': 'P300_Single_GEN2'}

# 2. 

Overall strategy:
Given N component names, \
and N-1:
    - volume fractions, 
    - np.linspace() arguments
    - concentration types
    
Calculate the volume fractions for all sample combinations. 

Regarding experiment_plan_dict
    - ['Component names'] corresponds to "chemical abbreviation" in the chemical inventory df.\
    This dictionary entry is a list of N components. The index of this list will correspond to\
    concentration types, and concentration linspace.
    - ['Concentration types'] can be volf, molarity, or mgpermL. 
    - ['Concentration linspace'] keyword arguments for linspace call for nth component.
   

# 3. 

generate_candidate_lattice_concentrations()
Calculates volume fractions of all components, including the Nth component.

Returns a dictionary of format:

"concentration type":numpy array of shape M x N 

where,
N is the number of components, M is the number of samples.

**NOTE** Mixture constrain is applied inside generate_candidate_lattice_concentrations. 
I.e., any sample for which the volume fraction of the last component is negative will be discarded due to being impossible.

In [114]:
exp_conc_dict = setup_functions.generate_candidate_lattice_concentrations(experiment_plan_dict, chemical_db_df)


# 4. 

Using the sample concentration arrays generated above, and chemical information of each component \
generate arrays of extensive properties. 

In [178]:
extensive_dict = setup_functions.get_extensive(exp_conc_dict, experiment_plan_dict, chemical_db_df)


No density input for chcl using density = 1.0


# Misc. Troubleshooting Below